In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
!cd ../; git clone -b master https://github.com/iamyyw/vilio.git
!cd ../vilio/py-bottom-up-attention; pip install -r requirements.txt
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!cd ../vilio/py-bottom-up-attention; python setup.py build develop

Cloning into 'vilio'...
remote: Enumerating objects: 2850, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 2850 (delta 16), reused 11 (delta 7), pack-reused 2819
Receiving objects: 100% (2850/2850), 10.62 MiB | 18.75 MiB/s, done.
Resolving deltas: 100% (1499/1499), done.
  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-cgcq3rc0
  Running command git clone -q https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-cgcq3rc0
     |████████████████████████████████| 753.4 MB 7.0 kB/s 
     |████████████████████████████████| 4.0 MB 59.1 MB/s 
     |████████████████████████████████| 636 kB 62.4 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5-py3-none-any.whl size=64506 sha256=228fe14d02a3ba32b9608c15449f50d0abd1b26b2c9a1c49abb17109fae6185c
  Stored in directory: /tmp/pip-ephem-wheel-cache-xk8y6x4z/wheels/24/1d/09/8167de727fe5b74f832b6fcb5d9069d8f03ca29f337bfe484d
Successfully bui

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%%writefile new_req.txt
sacremoses==0.0.43
pandas==1.1.3
regex==2020.4.4
h5py==2.10.0
filelock==3.0.10
scipy==1.4.1
sentencepiece~=0.1.91
matplotlib==3.2.1
# torch==1.6.0
tensorflow==2.3.1
tqdm==4.45.0
numpy==1.18.1
six==1.14.0
packaging==20.1
wandb==0.10.8
psutil==5.7.0
requests==2.23.0
pytorch_lightning==1.0.4
ImageHash==4.1.0
tokenizers~=0.9.2
transformers==3.5.1 # Required due to some imports in the files under src/vilio/transformers
torchvision==0.7.0
jieba==0.42.1
botocore==1.19.8
spacy==2.3.2
boto3==1.16.8
comet_ml==3.2.5
dataclasses==0.6
fairseq==0.9.0
ftfy==5.8
fugashi==1.0.5
ipadic==1.0.0
lmdb==1.0.0
Pillow==8.0.1
py3nvml==0.2.6
pydantic==1.7.2
pythainlp==2.2.4
PyYAML==5.3.1
scikit_learn==0.23.2
tensorboardX==2.1
timeout_decorator==0.4.1
torchcontrib==0.0.2

Writing new_req.txt


In [ ]:
import os, sys
import argparse
import random
import numpy as np
import torch
print(os.getcwd())
sys.path.insert(0, '../vilio')

/content


In [ ]:
data_path = r'"/content/drive/MyDrive/Colab Notebooks/hateful_meme/data"'

In [ ]:
#comment out if tsv is already generated
# !cd ../vilio/py-bottom-up-attention; python detectron2_mscoco_proposal_maxnms.py --dataroot $data_path --batchsize 4 --split img --weight vgattr --minboxes 72 --maxboxes 72

In [ ]:
# def get_args():
#     parser = argparse.ArgumentParser()

#     # Data Splits
#     parser.add_argument("--train", default=None)
#     parser.add_argument("--valid", default=None)
#     parser.add_argument("--test", default=None)

#     # Data-Type
#     parser.add_argument("--tsv", action='store_const', default=False, const=True, help='Whether to use tsv extraction (else lmdb)')
#     parser.add_argument("--num_features", type=int, default=100, help='How many features we have per img (e.g. 100, 80)')
#     parser.add_argument("--num_pos", type=int, default=4, help='How many position feats - 4 or 6')

#     # Training Hyper-parameters
#     parser.add_argument('--batchSize', dest='batch_size', type=int, default=8)
#     parser.add_argument('--optim', default='bert')
#     parser.add_argument('--lr', type=float, default=1e-05)
#     parser.add_argument('--epochs', type=int, default=5)
#     parser.add_argument('--dropout', type=float, default=0.1)
#     parser.add_argument('--seed', type=int, default=42, help='random seed')
#     parser.add_argument("--clip", type=float, default=5.0, help='Clip Grad Norm')
#     parser.add_argument("--wd", type=float, default=0.0, help='Weight Decay')
#     parser.add_argument("--swa", action='store_const', default=False, const=True)
#     parser.add_argument("--contrib", action='store_const', default=False, const=True)
#     parser.add_argument("--case", action='store_const', default=False, const=True)
#     parser.add_argument("--reg", action='store_const', default=False, const=True) # Applies Multi-sample dropout & Layeravg
#     parser.add_argument("--acc", type=int, default=1, help='Amount of accumulation steps for bigger batch size - make sure to adjust LR')
#     parser.add_argument('--subtrain', action='store_const', default=False, const=True)
#     parser.add_argument('--subtest', action='store_const', default=False, const=True)
#     parser.add_argument('--combine', action='store_const', default=False, const=True, help="Combine subtrained data; subtrain must be true")

#     # Debugging
#     parser.add_argument('--output', type=str, default='./data', help='Where ckpts, csvs shall be saved to')
#     parser.add_argument("--tqdm", action='store_const', default=False, const=True)
#     parser.add_argument("--topk", type=int, default=-1, help='For testing only load topk feats from tsv')
#     parser.add_argument("--exp", type=str, default="experiment", help="Name of experiment for csv's")

#     # Model Loading & Saving - Note: PATHS must be put in here! 
#     parser.add_argument('--model', type=str, default='X', help='Type of Model, X V O U D')
#     parser.add_argument("--tr", type=str, default="bert-base-uncased", help="Name of NLP transformer to be used")
#     parser.add_argument("--midsave", type=int, default=-1, help='Save a MID model after x steps')
#     parser.add_argument('--loadfin', type=str, default=None,
#                         help='Load the model (usually the fine-tuned model).')               
#     parser.add_argument('--loadpre', type=str, default=None,
#                         help='Load the pre-trained model.')
#     parser.add_argument("--fromScratch", dest='from_scratch', action='store_const', default=False, const=True,
#                         help='If none of the --load, --loadLXMERT, --loadLXMERTQA is set, '
#                              'the model would be trained from scratch. If --fromScratch is'
#                              ' not specified, the model would load BERT-pre-trained weights by'
#                              ' default. ')

#     # LXRT Model Config
#     # Note: LXRT = L, X, R (three encoders), Transformer
#     parser.add_argument("--llayers", default=9, type=int, help='Number of Language layers')
#     parser.add_argument("--xlayers", default=5, type=int, help='Number of CROSS-modality layers.')
#     parser.add_argument("--rlayers", default=5, type=int, help='Number of object Relationship layers.')

#     # Pre-training Config
#     parser.add_argument("--taskHM", dest='task_hm', action='store_const', default=False, const=True)
#     parser.add_argument("--taskMatched", dest='task_matched', action='store_const', default=False, const=True)
#     parser.add_argument("--taskMaskLM", dest='task_mask_lm', action='store_const', default=False, const=True)
#     parser.add_argument("--taskObjPredict", dest='task_obj_predict', action='store_const', default=False, const=True)
#     parser.add_argument("--taskQA", dest='task_qa', action='store_const', default=False, const=True)
#     parser.add_argument("--visualLosses", dest='visual_losses', default='obj,attr,feat', type=str)
#     parser.add_argument("--qaSets", dest='qa_sets', default=None, type=str)
#     parser.add_argument("--wordMaskRate", dest='word_mask_rate', default=0.15, type=float)
#     parser.add_argument("--objMaskRate", dest='obj_mask_rate', default=0.15, type=float)

#     # Training configuration
#     parser.add_argument("--multiGPU", action='store_const', default=False, const=True)
#     parser.add_argument("--numWorkers", dest='num_workers', default=4)

#     # Parse the arguments.
#     args = parser.parse_args([])

#     # Set seeds
#     torch.manual_seed(args.seed)
#     random.seed(args.seed)
#     np.random.seed(args.seed)

#     return args

In [ ]:
!pip install -r new_req.txt

     |████████████████████████████████| 883 kB 14.1 MB/s 
     |████████████████████████████████| 9.5 MB 15.8 MB/s 
     |████████████████████████████████| 679 kB 35.6 MB/s 
     |████████████████████████████████| 2.9 MB 62.6 MB/s 
     |████████████████████████████████| 1.2 MB 42.8 MB/s 
     |████████████████████████████████| 12.4 MB 23.0 MB/s 
     |████████████████████████████████| 320.4 MB 24 kB/s 
     |████████████████████████████████| 60 kB 8.7 MB/s 
     |████████████████████████████████| 20.1 MB 1.2 MB/s 
     |████████████████████████████████| 1.7 MB 40.6 MB/s 
     |████████████████████████████████| 449 kB 66.0 MB/s 
     |████████████████████████████████| 554 kB 54.5 MB/s 
     |████████████████████████████████| 291 kB 74.3 MB/s 
     |████████████████████████████████| 2.9 MB 42.3 MB/s 
     |████████████████████████████████| 1.3 MB 46.3 MB/s 
     |████████████████████████████████| 5.9 MB 7.4 MB/s 
     |████████████████████████████████| 6.7 MB 19.7 MB/s 
     |██████████

In [ ]:
#have to do it again...
data_path = r'"/content/drive/MyDrive/Colab Notebooks/hateful_meme/data"'

In [ ]:
!cp $data_path/dev_seen.jsonl ../vilio/data/dev_seen.jsonl
!cp $data_path/dev_unseen.jsonl ../vilio/data/dev_unseen.jsonl
!cp $data_path/test_seen.jsonl ../vilio/data/test_seen.jsonl
!cp $data_path/test_unseen.jsonl ../vilio/data/test_unseen.jsonl
!cp $data_path/train.jsonl ../vilio/data/train.jsonl
!cp $data_path/hm_vgattr3636.tsv ../vilio/data/HM_img.tsv

In [ ]:
%%writefile ../vilio/hm_inf.py
import collections
import os

from param import args

import numpy as np

from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data.dataloader import DataLoader

if args.tsv:
    from fts_tsv.hm_data_tsv import HMTorchDataset, HMEvaluator, HMDataset
else:
    from fts_lmdb.hm_data import HMTorchDataset, HMEvaluator, HMDataset

from src.vilio.transformers.optimization import AdamW, get_linear_schedule_with_warmup
from utils.pandas_scripts import clean_data

from entryU import ModelU
from entryX import ModelX
from entryV import ModelV
from entryD import ModelD
from entryO import ModelO

# Two different SWA Methods - https://pytorch.org/blog/pytorch-1.6-now-includes-stochastic-weight-averaging/
if args.swa:
    from torch.optim.swa_utils import AveragedModel, SWALR
    from torch.optim.lr_scheduler import CosineAnnealingLR

if args.contrib:
    from torchcontrib.optim import SWA


# Largely sticking to standards set in LXMERT here
DataTuple = collections.namedtuple("DataTuple", 'dataset loader evaluator')

def get_tuple(splits: str, bs:int, shuffle=False, drop_last=False) -> DataTuple:

    dset =  HMDataset(splits)
    
    tset = HMTorchDataset(splits)
    print('chkpot')
    evaluator = HMEvaluator(tset)

    data_loader = DataLoader(
        tset, batch_size=bs,
        shuffle=shuffle, num_workers=args.num_workers,
        drop_last=drop_last, pin_memory=True
    )

    return DataTuple(dataset=dset, loader=data_loader, evaluator=evaluator)

class HM:
    def __init__(self):
        
        if args.train is not None:
            self.train_tuple = get_tuple(
                args.train, bs=args.batch_size, shuffle=True, drop_last=False
            )

        if args.valid is not None:
            valid_bsize = 2048 if args.multiGPU else 50
            self.valid_tuple = get_tuple(
                args.valid, bs=valid_bsize,
                shuffle=False, drop_last=False
            )
        else:
            self.valid_tuple = None

        # Select Model, X is default
        if args.model == "X":
            self.model = ModelX(args)
        elif args.model == "V":
            self.model = ModelV(args)
        elif args.model == "U":
            self.model = ModelU(args)
        elif args.model == "D":
            self.model = ModelD(args)
        elif args.model == 'O':
            self.model = ModelO(args)
        else:
            print(args.model, " is not implemented.")

        # Load pre-trained weights from paths
        if args.loadpre is not None:
            self.model.load(args.loadpre)

        # GPU options
        if args.multiGPU:
            self.model.lxrt_encoder.multi_gpu()

        self.model = self.model.cuda()

        # Losses and optimizer
        self.logsoftmax = nn.LogSoftmax(dim=1)
        self.nllloss = nn.NLLLoss()

        if args.train is not None:
            batch_per_epoch = len(self.train_tuple.loader)
            self.t_total = int(batch_per_epoch * args.epochs // args.acc)
            print("Total Iters: %d" % self.t_total)

        def is_backbone(n):
            if "encoder" in n:
                return True
            elif "embeddings" in n:
                return True
            elif "pooler" in n:
                return True
            print("F: ", n)
            return False

        no_decay = ['bias', 'LayerNorm.weight']

        params = list(self.model.named_parameters())
        if args.reg:
            optimizer_grouped_parameters = [
                {"params": [p for n, p in params if is_backbone(n)], "lr": args.lr},
                {"params": [p for n, p in params if not is_backbone(n)], "lr": args.lr * 500},
            ]

            for n, p in self.model.named_parameters():
                print(n)

            self.optim = AdamW(optimizer_grouped_parameters, lr=args.lr)
        else:
            optimizer_grouped_parameters = [
                {'params': [p for n, p in params if not any(nd in n for nd in no_decay)], 'weight_decay': args.wd},
                {'params': [p for n, p in params if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
                ]

            self.optim = AdamW(optimizer_grouped_parameters, lr=args.lr)

        if args.train is not None:
            self.scheduler = get_linear_schedule_with_warmup(self.optim, self.t_total * 0.1, self.t_total)
        
        self.output = args.output
        os.makedirs(self.output, exist_ok=True)

        # SWA Method:
        if args.contrib:
            self.optim = SWA(self.optim, swa_start=self.t_total * 0.75, swa_freq=5, swa_lr=args.lr)

        if args.swa: 
            self.swa_model = AveragedModel(self.model)
            self.swa_start = self.t_total * 0.75
            self.swa_scheduler = SWALR(self.optim, swa_lr=args.lr)

    def train(self, train_tuple, eval_tuple):

        dset, loader, evaluator = train_tuple
        iter_wrapper = (lambda x: tqdm(x, total=len(loader))) if args.tqdm else (lambda x: x)

        print("Batches:", len(loader))

        self.optim.zero_grad()

        best_roc = 0.
        ups = 0
        
        total_loss = 0.

        for epoch in range(args.epochs):
            
            if args.reg:
                if args.model != "X":
                  pass
                    # print(self.model)
                    # print(self.model.model.layer_weights)

            id2ans = {}
            id2prob = {}

            for i, (ids, feats, boxes, sent, target) in iter_wrapper(enumerate(loader)):

                if ups == args.midsave:
                    self.save("MID")

                self.model.train()

                if args.swa:
                    self.swa_model.train()
                
                feats, boxes, target = feats.cuda(), boxes.cuda(), target.long().cuda()

                # Model expects visual feats as tuple of feats & boxes
                logit = self.model(sent, (feats, boxes))

                # Note: LogSoftmax does not change order, hence there should be nothing wrong with taking it as our prediction 
                # In fact ROC AUC stays the exact same for logsoftmax / normal softmax, but logsoftmax is better for loss calculation
                # due to stronger penalization & decomplexifying properties (log(a/b) = log(a) - log(b))
                logit = self.logsoftmax(logit)
                score = logit[:, 1]

                if i < 1:
                    print(logit[0, :].detach())
               
                # Note: This loss is the same as CrossEntropy (We splitted it up in logsoftmax & neg. log likelihood loss)
                loss = self.nllloss(logit.view(-1, 2), target.view(-1))

                # Scaling loss by batch size, as we have batches with different sizes, since we do not "drop_last" & dividing by acc for accumulation
                # Not scaling the loss will worsen performance by ~2abs%
                loss = loss * logit.size(0) / args.acc
                loss.backward()

                total_loss += loss.detach().item()

                # Acts as argmax - extracting the higher score & the corresponding index (0 or 1)
                _, predict = logit.detach().max(1)
                # Getting labels for accuracy
                for qid, l in zip(ids, predict.cpu().numpy()):
                    id2ans[qid] = l
                # Getting probabilities for Roc auc
                for qid, l in zip(ids, score.detach().cpu().numpy()):
                    id2prob[qid] = l

                if (i+1) % args.acc == 0:

                    nn.utils.clip_grad_norm_(self.model.parameters(), args.clip)

                    self.optim.step()

                    if (args.swa) and (ups > self.swa_start):
                        self.swa_model.update_parameters(self.model)
                        self.swa_scheduler.step()
                    else:
                        self.scheduler.step()
                    self.optim.zero_grad()

                    ups += 1

                    # Do Validation in between
                    if ups % 250 == 0: 
                        
                        log_str = "\nEpoch(U) %d(%d): Train AC %0.2f RA %0.4f LOSS %0.4f\n" % (epoch, ups, evaluator.evaluate(id2ans)*100, 
                        evaluator.roc_auc(id2prob)*100, total_loss)

                        # Set loss back to 0 after printing it
                        total_loss = 0.

                        if self.valid_tuple is not None:  # Do Validation
                            acc, roc_auc = self.evaluate(eval_tuple)
                            if roc_auc > best_roc:
                                best_roc = roc_auc
                                best_acc = acc
                                # Only save BEST when no midsave is specified to save space
                                #if args.midsave < 0:
                                #    self.save("BEST")

                            log_str += "\nEpoch(U) %d(%d): DEV AC %0.2f RA %0.4f \n" % (epoch, ups, acc*100.,roc_auc*100)
                            log_str += "Epoch(U) %d(%d): BEST AC %0.2f RA %0.4f \n" % (epoch, ups, best_acc*100., best_roc*100.)
    
                        print(log_str, end='')

                        with open(self.output + "/log.log", 'a') as f:
                            f.write(log_str)
                            f.flush()

        if (epoch + 1) == args.epochs:
            if args.contrib:
                self.optim.swap_swa_sgd()

        self.save("LAST" + args.train)

    def predict(self, eval_tuple: DataTuple, dump=None, out_csv=True):

        dset, loader, evaluator = eval_tuple
        id2ans = {}
        id2prob = {}

        for i, datum_tuple in enumerate(loader):

            ids, feats, boxes, sent = datum_tuple[:4]

            self.model.eval()

            if args.swa:
                self.swa_model.eval()

            with torch.no_grad():
                
                feats, boxes = feats.cuda(), boxes.cuda()
                logit = self.model(sent, (feats, boxes))

                # Note: LogSoftmax does not change order, hence there should be nothing wrong with taking it as our prediction
                logit = self.logsoftmax(logit)
                score = logit[:, 1]

                if args.swa:
                    logit = self.swa_model(sent, (feats, boxes))
                    logit = self.logsoftmax(logit)

                _, predict = logit.max(1)

                for qid, l in zip(ids, predict.cpu().numpy()):
                    id2ans[qid] = l

                # Getting probas for Roc Auc
                for qid, l in zip(ids, score.cpu().numpy()):
                    id2prob[qid] = l

        if dump is not None:
            if out_csv == True:
                evaluator.dump_csv(id2ans, id2prob, dump)
            else:
                evaluator.dump_result(id2ans, dump)

        return id2ans, id2prob

    def evaluate(self, eval_tuple: DataTuple, dump=None):
        """Evaluate all data in data_tuple."""
        id2ans, id2prob = self.predict(eval_tuple, dump=dump)

        acc = eval_tuple.evaluator.evaluate(id2ans)
        roc_auc = eval_tuple.evaluator.roc_auc(id2prob)
        print('acc:' + str(acc))
        print('roc_auc:' + str(roc_auc))
        return acc, roc_auc

    def save(self, name):
        if args.swa:
            torch.save(self.swa_model.state_dict(),
                    os.path.join(self.output, "%s.pth" % name))
        else:
            torch.save(self.model.state_dict(),
                    os.path.join(self.output, "%s.pth" % name))

    def load(self, path):
        print("Load model from %s" % path)
            
        state_dict = torch.load("%s" % path)
        new_state_dict = {}
        for key, value in state_dict.items():
            # N_averaged is a key in SWA models we cannot load, so we skip it
            if key.startswith("n_averaged"):
                print("n_averaged:", value)
                continue
            # SWA Models will start with module
            if key.startswith("module."):
                new_state_dict[key[len("module."):]] = value
            else:
                new_state_dict[key] = value
        state_dict = new_state_dict
        self.model.load_state_dict(state_dict)

def main():
    # Build Class
    hm = HM()

    # Load Model
    if args.loadfin is not None:
        hm.load(args.loadfin)

    # Train and/or Test:
    if args.train is not None:
        print('Splits in Train data:', hm.train_tuple.dataset.splits)
        if hm.valid_tuple is not None:
            print('Splits in Valid data:', hm.valid_tuple.dataset.splits)
        else:
            print("DO NOT USE VALIDATION")
        hm.train(hm.train_tuple, hm.valid_tuple)


        # If we also test afterwards load the last model
        if args.test is not None:
            hm.load(os.path.join(hm.output, "LAST" + args.train + ".pth"))

    if args.test is not None:
        # We can specify multiple test args e.g. test,test_unseen
        for split in args.test.split(","):
            # Anthing that has no labels:
            if 'test' in split:
                hm.predict(
                    get_tuple(split, bs=args.batch_size,
                            shuffle=False, drop_last=False),
                    dump=os.path.join(args.output, '{}_{}.csv'.format(args.exp, split))
                )
            # Anything else that has labels:
            elif 'dev' in split or 'valid' in split or 'train' in split:
                result = hm.evaluate(
                    get_tuple(split, bs=args.batch_size,
                            shuffle=False, drop_last=False),
                    dump=os.path.join(args.output, '{}_{}.csv'.format(args.exp, split))
                )
                print(result)
            # Same as test - assuming that it has no labels
            else:
                hm.predict(
                    get_tuple(split, bs=args.batch_size,
                            shuffle=False, drop_last=False),
                    dump=os.path.join(args.output, '{}_{}.csv'.format(args.exp, split))
                )

if __name__ == "__main__":
    main()

Writing ../vilio/hm_inf.py


In [ ]:
!cd ../vilio; python hm_inf.py --seed 42 --model X  --reg  \
--train train --valid dev_seen --test dev_seen --lr 0.00001 --batchSize 8 --tr bert-large-cased --tsv \
--num_features 36 --num_pos 6 --exp U36  --epochs 20 \
--output "/content/drive/MyDrive/pre-trained" 
# --loadpre "/content/drive/MyDrive/pre-trained/LASTtrain.pth"

2021-07-29 21:39:17.912180: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
comet_ml is installed but `COMET_API_KEY` is not set.
Load 8500 data from split(s) train.
Start to load Faster-RCNN detected objects from data/HM_img.tsv
Loaded 8500 images in file data/HM_img.tsv in 60 seconds.
Use 8500 data in torch dataset

chkpot
Load 500 data from split(s) dev_seen.
Start to load Faster-RCNN detected objects from data/HM_img.tsv
Loaded 500 images in file data/HM_img.tsv in 47 seconds.
Use 500 data in torch dataset

chkpot
LXRT encoder with 9 l_layers, 5 x_layers, and 5 r_layers.
Some weights of the model checkpoint at bert-large-cased were not used when initializing BertX: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.

In [ ]:
!cd ../vilio; python hm_inf.py --seed 42 --model X --reg \
--test test_seen --batchSize 8 --lr 0.00001 --tr bert-large-cased --tsv \
--num_features 36 --num_pos 6 --exp U36  \
--output "/content/drive/MyDrive/pre-trained" \
--loadfin "/content/drive/MyDrive/pre-trained/LASTtrain.pth"

2021-07-30 00:38:46.796249: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
comet_ml is installed but `COMET_API_KEY` is not set.
LXRT encoder with 9 l_layers, 5 x_layers, and 5 r_layers.
Some weights of the model checkpoint at bert-large-cased were not used when initializing BertX: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'bert.encoder.layer.9.attention.self.query.weight', 'bert.encoder.layer.9.attention.self.query.bias', 'bert.encoder.layer.9.attention.self.key.weight', 'bert.encoder.layer.9.attention.self.key.bias', 'bert.encoder.layer.9.attention.self.value.weight', 'bert.encoder.layer.9.attention.self.value.bias', 'bert.encoder.layer.9.attention.output.dense.weight',

In [ ]:
#convert test seen to be csv
import json, os
import pandas as pd

def convert_jsonl_to_df(file_nm):
  tmp = []
  for jline in open(file_nm, "r").read().split('\n'):
    if jline == '': continue
    d = json.loads(jline)
    d['id'] = int(d['id'])
    tmp.append(d)
  df = pd.DataFrame(tmp)
  return df

df_pred = pd.read_csv("/content/drive/MyDrive/pre-trained/U36_test_seen.csv")
df_test = convert_jsonl_to_df(r'/content/drive/MyDrive/Colab Notebooks/hateful_meme/data/test_seen.jsonl')[['id','label']]
df_test.rename(columns={'label':'test_label'},inplace=True)
df_combined = df_test.merge(df_pred,on='id',how='inner')

print(df_combined)

acc = sum(df_combined['label'] == df_combined['test_label'])/len(df_combined)
print('acc:' + str(acc))

import sklearn
from sklearn.metrics import roc_auc_score
roc_auc = sklearn.metrics.roc_auc_score(df_combined['label'], df_combined['test_label'])
print('roc_auc:' + str(roc_auc))

        id  test_label      proba  label
0    16395           1  -0.000002      1
1    37405           1  -9.354484      0
2    94180           1  -0.000001      1
3    54321           1 -17.564018      0
4    97015           1  -0.020172      1
..     ...         ...        ...    ...
995   3869           0 -20.492046      0
996  23817           0  -9.304251      0
997  56280           0 -20.431751      0
998  29384           0 -19.308590      0
999  34127           0 -20.380543      0

[1000 rows x 4 columns]
acc:0.624
roc_auc:0.6578813343519226
